In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1. We need to import libraries and data

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf
cf.go_offline()
%matplotlib inline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.linear_model import LassoCV
from sklearn import metrics 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, cross_val_score
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from scipy import stats
from scipy.stats import norm, skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

In [ ]:
train=pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
train.head()

In [ ]:
test=pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")
test.head()

In [ ]:
train.describe()
#Now we get an overall statistical picture of our data

In [ ]:
train.info()
#There are 1460 rows and 81 columns, 43 of which are objects, 35 of it are int64 and 3 of them are float64

In [ ]:
test.info()

2. Visualization of the Data

In [ ]:
train["SalePrice"].iplot(kind="hist")
#The target variable seems right-skewed distributed  mostly between 100 000 dolars and 200 000 dolars


In [ ]:
train["SalePrice"].mean(),train["SalePrice"].median(),train["SalePrice"].mode()
#Right skewed distribution means that the mean(180921) and median(163000) will be greater than the mode(140000) similar to this dataset. 
#Which means more houses were sold by less than the average price(180921).

In [ ]:
train["SalePrice"].iplot(kind="box") 

In [ ]:
plt.figure(figsize=(15,10))
train["SalePrice"].plot.box()
#The target variable has many outliers as seen both of box plots

In [ ]:
# The next step is to find out the correlation between target variable and other variables
#But this step only include numerical variables and non-numerical variables are excluded 
 
train.corr()["SalePrice"].sort_values(ascending = False)
#OverallQual column has the highest correlation point

In [ ]:
print(f"Skewness: {train['SalePrice'].skew()}")
print(f"Kurtosis: {train['SalePrice'].kurt()}")
figure = plt.figure(figsize=(15,10))
plt.subplot(1,2,1)
sns.distplot(train['SalePrice'] , fit=norm);
(mu, sigma) = norm.fit(train['SalePrice'])
plt.ylabel('Frequency')
plt.title('The Distribution of sale Prices of Houses')

plt.subplot(1,2,2)
stats.probplot(train['SalePrice'], plot=plt)
plt.show()

The Sale Prices of Houses is right skewed as it is seen above which means that  the mean greater than mode and median and median is greater than mode.We need to fix this skewness because many of the algorithms assume that the data science is normal and calculate various stats assuming this.We can fix this by using numpy.log1p() method normalise skweness by adding one(one added so that the zeros are being transformed to one as log of 0 is not defined) and taking natural log.  So the more the data is close to normal the more it fits the assumption.

In [ ]:
train[["OverallQual","SalePrice"]].scatter_matrix()
#The figures below there is a string correlation between the two 

In [ ]:
plt.figure(figsize=(20,15))
mask = np.zeros_like(train.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(train.corr(),annot=True,mask = mask,cmap="coolwarm")
#Here we visualize the correlation between all variables in the data set
plt.title("Heatmap of all the Features")

In [ ]:
train.head()

In [ ]:
train["SalePrice"]

In [ ]:
# Here we save target value for later and drop it from the train data set
y = train["SalePrice"]
train.drop('SalePrice', axis=1, inplace=True)
y

3. Dealing with the Missing Data

In [ ]:
#Now it is time to deal with the missing values in the columns of the data
train.isnull().sum().sort_values(ascending = False).head(35)
#There 35 columns which have missing values as seen below:

In [ ]:
test.isnull().sum().sort_values(ascending = False).head(35)

In [ ]:
plt.figure(figsize=(30,15))
sns.heatmap(train.isnull(),cmap="coolwarm")
# Now we visualize the missing values of the columns in our train data

In [ ]:
missing=train.isnull().sum().sort_values(ascending=False)
total=train.isnull().count().sort_values(ascending=False)
percentage=(train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
pd.concat([missing,total,percentage],axis=1,keys=["Missing","Total","Percentage"]).head(6)

In [ ]:
missing=test.isnull().sum().sort_values(ascending=False)
total=test.isnull().count().sort_values(ascending=False)
percentage=(test.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
pd.concat([missing,total,percentage],axis=1,keys=["Missing","Total","Percentage"]).head(6)

In [ ]:
#Some of the columns have many missing values, so it is better just drop them
#Because more than 80% of the column are missing in both train dataset and test dataset
train.drop(["PoolQC","MiscFeature","Alley","Fence"],axis=1,inplace=True)
test.drop(["PoolQC","MiscFeature","Alley","Fence"],axis=1,inplace=True)
plt.figure(figsize=(30,15))
sns.heatmap(train.isnull(),cmap="coolwarm")

In [ ]:
train.drop("FireplaceQu",axis=1,inplace=True) #This column also have many missing values. we will just get rid of them
test.drop("FireplaceQu",axis=1,inplace=True)
plt.figure(figsize=(30,15))
sns.heatmap(train.isnull(),cmap="coolwarm")


In [ ]:
"""Now the data is ready to ampute the missing values for the rest of columns
   We will segregate numeric and categoric columns in order to ampute them differently"""
numeric_cols = train.select_dtypes(['float64','int64']).columns
categoric_cols = train.select_dtypes('object').columns

numeric = train[numeric_cols]
categoric = train[categoric_cols]

def mean_imputation(numeric):
    """
    Filling the missing values with the mean of the categorical columns
    """
    for col in numeric.columns:
        mean = numeric[col].mean()
        numeric[col] = numeric[col].fillna(mean)
    return numeric

mean_imp = mean_imputation(numeric)

def mode_imputation(categoric):
    """
    Filling the missing values with the mode of the categorical columns
    """
    for col in categoric.columns:
        mode = categoric[col].mode().iloc[0]
        categoric[col] = categoric[col].fillna(mode)
    return categoric

mode_imp = mode_imputation(categoric)
'''We will concatenate the mean and mode imputed columns together'''
train = pd.concat([mean_imp, mode_imp], axis = 1)
train


In [ ]:
numeric_cols = test.select_dtypes(['float64','int64']).columns
categoric_cols = test.select_dtypes('object').columns

numeric = test[numeric_cols]
categoric = test[categoric_cols]

def mean_imputation(numeric):
    """
    Filling the missing values with the mean of the categorical columns
    """
    for col in numeric.columns:
        mean = numeric[col].mean()
        numeric[col] = numeric[col].fillna(mean)
    return numeric

mean_imp = mean_imputation(numeric)

def mode_imputation(categoric):
    """
    Filling the missing values with the mode of the categorical columns
    """
    for col in categoric.columns:
        mode = categoric[col].mode().iloc[0]
        categoric[col] = categoric[col].fillna(mode)
    return categoric

mode_imp = mode_imputation(categoric)
'''We will concatenate the mean and mode imputed columns together'''
test = pd.concat([mean_imp, mode_imp], axis = 1)
test

In [ ]:
train.isnull().sum().sort_values(ascending = False).head()
#Now there is not any column with missing data in our train dataset

In [ ]:
test.isnull().sum().sort_values(ascending = False).head()
#Now there is not any column with missing data in our test dataset

In [ ]:
train.info()
#Now we have 75 columns and 1460 rows without any missing valued column in the train  data set

In [ ]:
test.info()
#Now we have 74 columns and 1459 rows without any missing valued column


In [ ]:
#In order to be more precise, we need to convert floats into integers,because all floats have not any data after "."
for col in train.select_dtypes(include="float64").columns:
    train[col]=train[col].astype(np.int64)
    

In [ ]:
# we will do the same for the train data set as follows:
for col in test.select_dtypes(include="float64").columns:
    test[col]=train[col].astype(np.int64)


In [ ]:
train.select_dtypes(include="object").shape

In [ ]:
test.select_dtypes(include="object").shape

#Now both train and test data set have been syncronized in terms of types of data

Machine learning algorithms cannot run with categorical columns so we need to make them numerical:
We will use Ordinal Encoder for this purpose:
OrdinalEncoder(*, categories='auto', dtype=<class 'numpy.float64'>) |
| Encode categorical features as an integer array. |
| The input to this transformer should be an array-like of integers or | strings, denoting the values taken on by categorical (discrete) features. | The features are converted to ordinal integers. This results in | a single column of integers (0 to n_categories - 1) per feature.

In [ ]:
categoric_cols = train.select_dtypes('object').columns
numeric_cols=train.select_dtypes('int64').columns
categoric = train[categoric_cols]
numeric=train[numeric_cols]
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
for col in categoric.columns:
    categoric[col] = pd.DataFrame(ordinal_encoder.fit_transform(categoric)) #This returns a numpy array
categoric

In [ ]:
# we will do the same for our test data set:
test_categoric_cols = test.select_dtypes('object').columns
test_numeric_cols=test.select_dtypes('int64').columns
test_categoric = test[test_categoric_cols]
test_numeric=test[test_numeric_cols]
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
for col in test_categoric.columns:
    test_categoric[col] = pd.DataFrame(ordinal_encoder.fit_transform(test_categoric)) #This returns a numpy array
test_categoric

In [ ]:
#Now we will concetanate the previous numerical variables with the converted categorical variables
train_ready = pd.concat([numeric, categoric], axis = 1)
test_ready = pd.concat([test_numeric, test_categoric], axis = 1)

In [ ]:
train_ready.info()

In [ ]:
test_ready.info()

5.Splitting the Data and Training the Model

In [ ]:
X=train_ready # here we assing train data set as our X value for the algorithm
X.shape,y.shape

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.4,random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
lm=LinearRegression()
lm.fit(X_train,y_train)

In [ ]:
#In order to evaluate the performance we need to predict the test data and compare the predictions with the actual test data values
predictions=lm.predict(X_test)
#here we predict the y_test values from X_test data according to the our trained Linear Regression data
predictions



In [ ]:
# here I will visualize the real test values(y_test) versus the predicted values.
sns.scatterplot(y_test,predictions)
#It seems that our linear regression model predict ver well

In [ ]:
# We will evaluate our model performance by calculating the residual sum of squares and the explained variance score
from sklearn import metrics
print("MAE:",metrics.mean_absolute_error(y_test,predictions))
print ("MSE:",metrics.mean_squared_error(y_test,predictions))
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test,predictions)))

In [ ]:
#Evaluation of  the explained variance score (R^2)
metrics.explained_variance_score(y_test,predictions) #This shows our model predict %79.9 of the target correctly

In [ ]:
#Now we will visualize the differences between our predictions and actual y test data
sns.distplot(y_test-predictions,bins=50) #this figure also proves that our model fits very good
#There is no huge differences between our predictions and actual y data

In [ ]:
cdf=pd.DataFrame(lm.coef_,X.columns,columns=["Coefficients"])
cdf["Coefficients"].sort_values(ascending=False)

In [ ]:
#we se that our model predict good, we will predict the test data for this time
predictions=lm.predict(test_ready)

In [ ]:
predictions=pd.DataFrame(predictions)

In [ ]:
submit=pd.concat([test['Id'],predictions],axis=1)
submit.columns=['Id','SalePrice']
len(submit.isnull())

In [ ]:
submit.to_csv("submission.csv", index = False)